In [626]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
load_dotenv()

True

In [627]:
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs((y_true - y_pred))) / np.sum(np.abs(y_true))

In [628]:
host = os.getenv("HOST")
dbname = os.getenv("DATABASE_NAME")
user = os.getenv("USERNAME")
password = os.getenv("DATABASE_PASSWORD")

conn_string = "host={0} user={1} dbname={2} password={3}".format(host, user, dbname, password)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

In [629]:
query = "SELECT * FROM core_sale;"
df_salesmans = pd.read_sql(query, conn)

/home/francisco/workspace/lambda-system/machine-learning-dicascada/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [630]:
df_sales = df_salesmans.copy()


In [631]:
df_sales_copy = df_sales.copy()

In [632]:
df_sales_copy["year"] = df_sales_copy.apply(lambda row : row["date"].year, axis=1)
df_sales_copy["month"] = df_sales_copy.apply(lambda row : row["date"].month, axis=1)
df_sales_copy["day"] = df_sales_copy.apply(lambda row : row["date"].day, axis=1)
df_sales_copy.drop(columns=["date"], inplace=True)
df_sales_copy.drop(columns=["description"], inplace=True)
df_sales_copy.drop(columns=["status"], inplace=True)
df_sales_machine_learning = df_sales_copy.copy()

In [633]:
df_sales_copy.set_index(['year', 'month', 'day'], inplace=True)

In [634]:
df_groupby_sales = df_sales_copy.groupby(["year", "month"])
sales_data = {
    'income': df_groupby_sales['income'].sum(),
    'count': df_groupby_sales['id'].count()
}

df_sales_per_months = pd.DataFrame(sales_data)
df_sales_per_months

income  count
year month                  
2018 11      52361.97    144
     12      44421.13    138
2019 1       76548.24    149
     2       14674.46    106
     3       48730.99     71
     4      182488.12    201
     5      171914.80    272
     6      189254.46    222
     7      242337.45    230
     8      114204.27    236
     9       86824.01    273
     10     121750.63    276
     11     210125.16    566
     12     163990.10    456
2020 1      169677.59    498
     2      142474.62    421
     3      384130.64    506
     4      176819.59    383
     5      184797.90    379
     6      167705.86    575
     7      219422.97    649
     8      257356.98    634
     9      329349.40    742
     10     189891.24    702
     11     237440.65    666
     12     231967.06    572
2021 1      158706.08    595
     2      257297.47    872
     3      192503.59    777
     4      206944.43    710
     5      166545.74    739
     6      166283.13    745
     7      158943.72    876
     8      138120.65    816
     9      175135.81    908
     10     279933.26   1114
     11     272847.04   1106
     12     222096.95    838
2022 1      260265.91   1011
     2      160516.88    491

In [635]:
df_groupby_sales_year = df_sales_copy.groupby(["year"])
sales_data = {
    'income': df_groupby_sales_year['income'].sum(),
    'count': df_groupby_sales_year['id'].count()
}

df_sales_per_year = pd.DataFrame(sales_data)

In [636]:
df_groupby_sales_day = df_sales_copy.groupby(["year", "month","day"])
sales_data = {
    'income': df_groupby_sales_day['income'].sum(),
    'count': df_groupby_sales_day['id'].count()
}

df_sales_per_day = pd.DataFrame(sales_data)

In [637]:
df_sales_per_day

income  count
year month day                 
2018 11    12    6697.59     12
           13    4446.46     24
           14    1062.53      7
           15    1079.60      2
           16    3495.26     11
...                  ...    ...
2022 2     7    11799.68     25
           8    16709.13     71
           9    11288.48     50
           10    9933.31     50
           11    4762.40     37

[814 rows x 2 columns]

In [638]:

df_sales_per_months

income  count
year month                  
2018 11      52361.97    144
     12      44421.13    138
2019 1       76548.24    149
     2       14674.46    106
     3       48730.99     71
     4      182488.12    201
     5      171914.80    272
     6      189254.46    222
     7      242337.45    230
     8      114204.27    236
     9       86824.01    273
     10     121750.63    276
     11     210125.16    566
     12     163990.10    456
2020 1      169677.59    498
     2      142474.62    421
     3      384130.64    506
     4      176819.59    383
     5      184797.90    379
     6      167705.86    575
     7      219422.97    649
     8      257356.98    634
     9      329349.40    742
     10     189891.24    702
     11     237440.65    666
     12     231967.06    572
2021 1      158706.08    595
     2      257297.47    872
     3      192503.59    777
     4      206944.43    710
     5      166545.74    739
     6      166283.13    745
     7      158943.72    876
     8      138120.65    816
     9      175135.81    908
     10     279933.26   1114
     11     272847.04   1106
     12     222096.95    838
2022 1      260265.91   1011
     2      160516.88    491

In [639]:

df_sales_per_year

,income,count
year,,
2018,96783.10,282
2019,1622842.69,3058
2020,2691034.50,6727
2021,2395357.87,10096
2022,420782.79,1502


In [640]:
df_sales_machine_learning = df_sales_per_day.copy()

In [641]:
split_point = 2022
df_sales_machine_learning.reset_index(inplace=True)

df_sales_machine_learning
data_train = df_sales_machine_learning.loc[df_sales_machine_learning['year'] < split_point].copy()
data_valid = df_sales_machine_learning.loc[df_sales_machine_learning['year'] >= split_point].copy()

In [642]:
month_data_train = data_train.copy()
month_data_valid = data_valid.copy()
month_data_train["sales_next_month_in_day"] = data_train.groupby("day")["income"].shift(-1)
month_data_train["sales_next_month_in_day_count"] = data_train.groupby("day")["count"].shift(-1)
month_data_valid["sales_next_month_in_day"] = data_valid.groupby("day")["income"].shift(-1)
month_data_valid["sales_next_month_in_day_count"] = data_valid.groupby("day")["count"].shift(-1)

month_data_train[month_data_train["day"] == 12]

,year,month,day,income,count,sales_next_month_in_day,sales_next_month_in_day_count
0,2018,11,12,6697.59,12,21.49,1.0
20,2018,12,12,21.49,1,1550.40,12.0
49,2019,2,12,1550.40,12,1157.94,5.0
75,2019,4,12,1157.94,5,610.83,2.0
115,2019,6,12,610.83,2,279.29,4.0
138,2019,7,12,279.29,4,2752.49,11.0
159,2019,8,12,2752.49,11,1177.09,17.0
180,2019,9,12,1177.09,17,980.13,2.0
201,2019,10,12,980.13,2,27376.24,16.0
223,2019,11,12,27376.24,16,18656.84,33.0


In [643]:
data_train_day = data_train.copy()
data_valid_day = data_valid.copy()
data_train_day["sales_next_day"] = data_train["income"].shift(-1)
data_train_day["sales_next_day_count"] = data_train["count"].shift(-1)
data_valid_day["sales_next_day"] = data_valid["income"].shift(-1)
data_valid_day["sales_next_day_count"] = data_valid["count"].shift(-1)
data_train_day

,year,month,day,income,count,sales_next_day,sales_next_day_count
0,2018,11,12,6697.59,12,4446.46,24.0
1,2018,11,13,4446.46,24,1062.53,7.0
2,2018,11,14,1062.53,7,1079.60,2.0
3,2018,11,15,1079.60,2,3495.26,11.0
4,2018,11,16,3495.26,11,1057.18,3.0
...,...,...,...,...,...,...,...
781,2021,12,24,6637.56,16,3744.06,20.0
782,2021,12,27,3744.06,20,9308.59,59.0
783,2021,12,28,9308.59,59,5518.38,38.0
784,2021,12,29,5518.38,38,2697.19,24.0


In [644]:
data_train_day.dropna(inplace=True)
data_train_day["lag_sales_next_day"] = data_train_day["income"].shift(1)
data_train_day["lag_sales_next_day_count"] = data_train_day["count"].shift(1)
data_train_day

,year,month,day,income,count,sales_next_day,sales_next_day_count,lag_sales_next_day,lag_sales_next_day_count
0,2018,11,12,6697.59,12,4446.46,24.0,NaN,NaN
1,2018,11,13,4446.46,24,1062.53,7.0,6697.59,12.0
2,2018,11,14,1062.53,7,1079.60,2.0,4446.46,24.0
3,2018,11,15,1079.60,2,3495.26,11.0,1062.53,7.0
4,2018,11,16,3495.26,11,1057.18,3.0,1079.60,2.0
...,...,...,...,...,...,...,...,...,...
780,2021,12,23,14936.40,69,6637.56,16.0,22189.92,103.0
781,2021,12,24,6637.56,16,3744.06,20.0,14936.40,69.0
782,2021,12,27,3744.06,20,9308.59,59.0,6637.56,16.0
783,2021,12,28,9308.59,59,5518.38,38.0,3744.06,20.0


In [645]:
data_valid_day["lag_sales_next_day"] = data_valid_day["income"].shift(1)
data_valid_day["lag_sales_next_day_count"] = data_valid_day["count"].shift(1)
data_valid_day

,year,month,day,income,count,sales_next_day,sales_next_day_count,lag_sales_next_day,lag_sales_next_day_count
786,2022,1,5,6478.88,38,17035.09,90.0,NaN,NaN
787,2022,1,6,17035.09,90,4542.59,31.0,6478.88,38.0
788,2022,1,7,4542.59,31,22410.47,69.0,17035.09,90.0
789,2022,1,10,22410.47,69,22752.23,64.0,4542.59,31.0
790,2022,1,11,22752.23,64,5830.05,11.0,22410.47,69.0
791,2022,1,12,5830.05,11,8913.54,59.0,22752.23,64.0
792,2022,1,13,8913.54,59,3454.24,27.0,5830.05,11.0
793,2022,1,14,3454.24,27,8167.05,28.0,8913.54,59.0
794,2022,1,17,8167.05,28,23315.77,69.0,3454.24,27.0
795,2022,1,18,23315.77,69,30300.34,88.0,8167.05,28.0


In [646]:
data_valid_day["diff_sales_next_day"] = data_valid_day["income"].diff(1)
data_valid_day["diff_sales_next_day_count"] = data_valid_day["count"].diff(1)
data_valid_day

,year,month,day,income,count,sales_next_day,sales_next_day_count,lag_sales_next_day,lag_sales_next_day_count,diff_sales_next_day,diff_sales_next_day_count
786,2022,1,5,6478.88,38,17035.09,90.0,NaN,NaN,NaN,NaN
787,2022,1,6,17035.09,90,4542.59,31.0,6478.88,38.0,10556.21,52.0
788,2022,1,7,4542.59,31,22410.47,69.0,17035.09,90.0,-12492.50,-59.0
789,2022,1,10,22410.47,69,22752.23,64.0,4542.59,31.0,17867.88,38.0
790,2022,1,11,22752.23,64,5830.05,11.0,22410.47,69.0,341.76,-5.0
791,2022,1,12,5830.05,11,8913.54,59.0,22752.23,64.0,-16922.18,-53.0
792,2022,1,13,8913.54,59,3454.24,27.0,5830.05,11.0,3083.49,48.0
793,2022,1,14,3454.24,27,8167.05,28.0,8913.54,59.0,-5459.30,-32.0
794,2022,1,17,8167.05,28,23315.77,69.0,3454.24,27.0,4712.81,1.0
795,2022,1,18,23315.77,69,30300.34,88.0,8167.05,28.0,15148.72,41.0


In [647]:
data_train_day["diff_sales_next_day"] = data_train_day["income"].diff(1)
data_train_day["diff_sales_next_day_count"] = data_train_day["count"].diff(1)
data_train_day

,year,month,day,income,count,sales_next_day,sales_next_day_count,lag_sales_next_day,lag_sales_next_day_count,diff_sales_next_day,diff_sales_next_day_count
0,2018,11,12,6697.59,12,4446.46,24.0,NaN,NaN,NaN,NaN
1,2018,11,13,4446.46,24,1062.53,7.0,6697.59,12.0,-2251.13,12.0
2,2018,11,14,1062.53,7,1079.60,2.0,4446.46,24.0,-3383.93,-17.0
3,2018,11,15,1079.60,2,3495.26,11.0,1062.53,7.0,17.07,-5.0
4,2018,11,16,3495.26,11,1057.18,3.0,1079.60,2.0,2415.66,9.0
...,...,...,...,...,...,...,...,...,...,...,...
780,2021,12,23,14936.40,69,6637.56,16.0,22189.92,103.0,-7253.52,-34.0
781,2021,12,24,6637.56,16,3744.06,20.0,14936.40,69.0,-8298.84,-53.0
782,2021,12,27,3744.06,20,9308.59,59.0,6637.56,16.0,-2893.50,4.0
783,2021,12,28,9308.59,59,5518.38,38.0,3744.06,20.0,5564.53,39.0


In [648]:
y_pred = data_train_day["income"]
y_true = data_train_day['sales_next_day']

In [649]:
mape(y_true, y_pred)

9.039333582313759

In [650]:
wmape(y_true, y_pred)

0.9307289036969174

In [651]:
data_train_day.tail()

,year,month,day,income,count,sales_next_day,sales_next_day_count,lag_sales_next_day,lag_sales_next_day_count,diff_sales_next_day,diff_sales_next_day_count
780,2021,12,23,14936.40,69,6637.56,16.0,22189.92,103.0,-7253.52,-34.0
781,2021,12,24,6637.56,16,3744.06,20.0,14936.40,69.0,-8298.84,-53.0
782,2021,12,27,3744.06,20,9308.59,59.0,6637.56,16.0,-2893.50,4.0
783,2021,12,28,9308.59,59,5518.38,38.0,3744.06,20.0,5564.53,39.0
784,2021,12,29,5518.38,38,2697.19,24.0,9308.59,59.0,-3790.21,-21.0


In [652]:
features = ["income", "count", "lag_sales_next_day", "lag_sales_next_day_count", "diff_sales_next_day", "diff_sales_next_day_count"]

In [657]:
imputer = SimpleImputer()
Xtr = imputer.fit_transform(data_train_day[features])
ytr = data_train_day['sales_next_day']


model = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
model.fit(Xtr, ytr)


RandomForestRegressor(n_jobs=6, random_state=0)

In [658]:
Xval = imputer.transform(data_valid_day[features])
yval = data_valid_day['sales_next_day']

p = model.predict(Xval)

df_forest = pd.DataFrame({"predicted": p, "true": yval})
df_forest

,predicted,true
786,7141.7398,17035.09
787,5801.9262,4542.59
788,7079.8645,22410.47
789,10985.9003,22752.23
790,11497.9482,5830.05
791,6658.5190,8913.54
792,6820.7832,3454.24
793,4998.4236,8167.05
794,9633.9118,23315.77
795,9250.4975,30300.34


In [659]:
data_valid_day
mape(yval, p)

0.7268474176165012

In [660]:
wmape(yval, p)

0.553518081931691